In [2]:
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, PooledFlairEmbeddings
from typing import List
from flair.visual.training_curves import Plotter

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [4]:
# initialize embeddings. This takes time to load.
embedding_types: List[TokenEmbeddings] = [
    # GloVe embeddings for arabic
    WordEmbeddings('ar'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

2019-04-13 18:20:25,555 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/ar-wiki-fasttext-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmpzp4frq7k


100%|██████████| 733171328/733171328 [06:49<00:00, 1789139.10B/s] 

2019-04-13 18:27:16,060 copying /tmp/tmpzp4frq7k to cache at /home/jupyter/.flair/embeddings/ar-wiki-fasttext-300d-1M.vectors.npy


2019-04-13 18:27:16,827 removing temp file /tmp/tmpzp4frq7k
2019-04-13 18:27:17,525 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4/ar-wiki-fasttext-300d-1M not found in cache, downloading to /tmp/tmp875hc_rx


100%|██████████| 26704903/26704903 [00:02<00:00, 9335785.91B/s] 

2019-04-13 18:27:21,064 copying /tmp/tmp875hc_rx to cache at /home/jupyter/.flair/embeddings/ar-wiki-fasttext-300d-1M
2019-04-13 18:27:21,093 removing temp file /tmp/tmp875hc_rx


2019-04-13 18:27:21,184 this function is deprecated, use smart_open.open instead


In [6]:

# 1. get the corpus
columns = {0: 'text', 1: 'ner'}
data_folder = '/home/jupyter/data_ner/'
corpus: TaggedCorpus = NLPTaskDataFetcher.load_column_corpus(data_folder, columns, 
  train_file='train.txt', 
  dev_file='dev.txt')

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


# initialize sequence tagger
from flair.models import SequenceTagger

# have a relatively small hidden_size
tagger: SequenceTagger = SequenceTagger(hidden_size=64,
                                        dropout = 0.2,
                                        rnn_layers = 2,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)# initialize trainer
# 6. Initialize trainer
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

from flair.optim import AdamW
# optimizer = optimizer(momentum=0.9, dampening=0.9)
trainer: ModelTrainer = ModelTrainer(tagger, corpus, optimizer=AdamW)


2019-04-13 18:31:16,855 Reading data from /home/jupyter/data_ner
2019-04-13 18:31:16,857 Train: /home/jupyter/data_ner/train.txt
2019-04-13 18:31:16,857 Dev: /home/jupyter/data_ner/dev.txt
2019-04-13 18:31:16,858 Test: None


In [ ]:
import os
# 7. start training
result_folder = '/home/jupyter/result_64_2_l01_w01_d02_adamw/'
trainer.train(result_folder,
              EvaluationMetric.MICRO_F1_SCORE,
              learning_rate=0.1,
              mini_batch_size=32,
              weight_decay = 0.1,
              max_epochs=200,
              checkpoint=False)
from flair.visual.training_curves import Plotter
plotter = Plotter()
plotter.plot_training_curves(os.path.join(result_folder, 'loss.tsv'))
plotter.plot_weights(os.path.join(result_folder, 'weights.tsv'))

2019-04-13 18:47:57,765 ----------------------------------------------------------------------------------------------------
2019-04-13 18:47:57,766 Evaluation method: MICRO_F1_SCORE
2019-04-13 18:47:57,782 ----------------------------------------------------------------------------------------------------
2019-04-13 18:47:58,003 epoch 1 - iter 0/38 - loss 12.88837433
2019-04-13 18:47:58,872 epoch 1 - iter 3/38 - loss 28.40017533
2019-04-13 18:47:59,777 epoch 1 - iter 6/38 - loss 23.35679831
2019-04-13 18:48:00,761 epoch 1 - iter 9/38 - loss 21.71760406
2019-04-13 18:48:01,506 epoch 1 - iter 12/38 - loss 19.08509687
2019-04-13 18:48:02,890 epoch 1 - iter 15/38 - loss 20.15971559
2019-04-13 18:48:04,171 epoch 1 - iter 18/38 - loss 19.49532649
2019-04-13 18:48:05,290 epoch 1 - iter 21/38 - loss 18.65723636
2019-04-13 18:48:06,219 epoch 1 - iter 24/38 - loss 18.13274155
2019-04-13 18:48:07,199 epoch 1 - iter 27/38 - loss 17.32379975
2019-04-13 18:48:08,029 epoch 1 - iter 30/38 - loss 16.8